In [1]:
import os ,sys 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/Users/barradd/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
seed = 101
tf.set_random_seed(seed)
np.random.seed(seed)

In [3]:
df = pd.read_csv('SubCh1_TrainingData.csv')
df.set_index("Sample_Name",inplace=True)
df_edit = pd.concat( [df, pd.get_dummies(df["Treatment"]),
                      pd.get_dummies( df["Timepoint"])]
                         ,axis=1,sort=True )
df = df_edit.drop(["Treatment","Timepoint","BioRep","Isolate","DHA_IC50"],axis=1)

In [4]:
df_subch2 = pd.read_csv("SubCh2_TestData.csv")
df_subch2.set_index("Sample_Names",inplace=True)
df_edit = pd.concat( [df_subch2, 
                      pd.get_dummies(df_subch2["Treatment"]),
                      pd.get_dummies( df_subch2["Timepoint"])]
                         ,axis=1,sort=True )
df_subch2= df_edit.drop(['Treatment','Timepoint','BioRep',
                         'Country','Isolate','ClearanceRate','Asexual_Stage'],axis=1)
# 'MAL1.465720.465873.bc..rv..',
# 'MAL1.48287.48430....kr...can',
# 'MAL1.562126.562246.bc..r...',
# 'MAL1.59416.59687....kr...can',
# 'MAL1.616633.616682.......y.',
# 'MAL10.1139073.1139243....kr',
# 'MAL10.258733.258825.b...r..',
# 'MAL10.271753.271892.bcgkrvy'
#                           ],axis=1)

In [5]:
set(df_subch2.columns.values.tolist()) -set(df.columns.values.tolist())

set()

In [6]:
features_missisng = list(set(df.columns.values.tolist()) - set(df_subch2.columns.values.tolist()))

In [7]:
df.shape,df_subch2.shape

((272, 5544), (288, 4964))

In [8]:
df.shape[1]-df_subch2.shape[1]

580

In [9]:
len(features_missisng)

580

In [10]:
df[["MAL1.465720.465873.bc..rv..","MAL10.1139073.1139243....kr","PF3D7_1470600","PF3D7_1480100"]].describe()

,MAL1.465720.465873.bc..rv..,MAL10.1139073.1139243....kr,PF3D7_1470600,PF3D7_1480100
count,272.000000,272.000000,272.000000,272.000000
mean,0.125273,0.298666,0.246266,-1.746997
std,0.542060,0.425972,0.507507,0.800431
min,-1.250413,-0.694826,-0.892560,-3.707360
25%,-0.233872,-0.019275,-0.137358,-2.294710
50%,0.130822,0.304761,0.145848,-1.822472
75%,0.471552,0.616233,0.631874,-1.333393
max,1.530355,1.371144,1.625414,1.015633


In [11]:
for m in features_missisng: 
    df_subch2[m] = 0.0

In [12]:
df.shape,df_subch2.shape

((272, 5544), (288, 5544))

In [13]:
sc = StandardScaler()

In [14]:
Xsc = sc.fit(df)

/Users/barradd/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [15]:
X_test = sc.transform(df_subch2)

/Users/barradd/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [16]:
X_test.shape

(288, 5544)

In [17]:
my_model = tf.keras.models.load_model("my_TF1.9_model_copy_73acc.h5")

In [ ]:
my_model.tr

In [18]:
test_predictions = my_model.predict(X_test).flatten()

In [19]:
test_predictions.max(),test_predictions.min(),test_predictions.std(),test_predictions.mean(),test_predictions.max()-test_predictions.min()

(1.9806294, 1.5800676, 0.071577504, 1.785892, 0.4005618)

In [ ]:
test_predictions

In [20]:
my_limit = test_predictions.std()+test_predictions.mean()

In [21]:
type(test_predictions),test_predictions.shape

(numpy.ndarray, (288,))

In [22]:
test_predictions[0]

1.7754272

In [25]:
#https://malariajournal.biomedcentral.com/track/pdf/10.1186/1475-2875-12-408
my_pred = [] 
for x in test_predictions:
    if x < test_predictions.mean():
#     if x < my_limit : 
        my_pred.append ((x, "SLOW"))
    else :
        my_pred.append ((x,"FAST"))

In [26]:
my_pred[:10]

[(1.7754272, 'SLOW'),
 (1.7306185, 'SLOW'),
 (1.7019646, 'SLOW'),
 (1.8477347, 'FAST'),
 (1.8282176, 'FAST'),
 (1.8545588, 'FAST'),
 (1.8060082, 'FAST'),
 (1.863467, 'FAST'),
 (1.7677963, 'SLOW'),
 (1.8336699, 'FAST')]

In [27]:
pred_prev = pd.DataFrame(my_pred,columns=[ u'Probability',u'Predicted_Categorical_Clearance'])

In [28]:
dftemp = pd.read_csv("SubCh2_TestData.csv")

In [29]:
pred_prev.shape , dftemp.shape

((288, 2), (288, 4968))

In [30]:
dftemp.columns

Index([u'Sample_Names', u'Country', u'Asexual_Stage', u'Isolate', u'Timepoint',
       u'Treatment', u'BioRep', u'MAL1.465720.465873.bc..rv..',
       u'MAL1.48287.48430....kr...can', u'MAL1.562126.562246.bc..r...',
       ...
       u'PF3D7_1478800', u'PF3D7_1479000', u'PF3D7_1479200', u'PF3D7_1479400',
       u'PF3D7_1479500', u'PF3D7_1479600', u'PF3D7_1479900', u'PF3D7_1480000',
       u'PF3D7_1480100', u'ClearanceRate'],
      dtype='object', length=4968)

In [32]:
second = pd.concat([dftemp[u'Isolate'],pred_prev],axis=1)

In [54]:
submision = []
for m in second["Isolate"].unique():
    std = second[second["Isolate"] == m]["Probability"].std()
    mean = second[second["Isolate"] == m]["Probability"].mean()
#     print m, mean,std, mean > test_predictions.mean() 
    maximun = second[second["Isolate"] == m]["Probability"].max()
    minimun = second[second["Isolate"] == m]["Probability"].min()
#     if mean > test_predictions.mean()  :
    if maximun-std > test_predictions.mean()  :
        submision.append((m, "FAST", mean  ))
    else:
        submision.append((m, "SLOW", mean  ))

In [55]:
final_df = pd.DataFrame(submision,columns= [u'Isolate' , 
                                            u'Predicted_Categorical_Clearance' , 
                                            u'Probability' ])

In [56]:
final_df.groupby("Predicted_Categorical_Clearance").nunique()

,Isolate,Predicted_Categorical_Clearance,Probability
Predicted_Categorical_Clearance,,,
FAST,27,1,27
SLOW,5,1,5


In [57]:
final_df.to_csv("SubCh2_Submission_DBB_from_model_SubCh1.txt",sep="\t",index=False)